In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.utils.prune as prune
import json
import time
from sklearn.metrics import mean_absolute_error

from agemodel import AgePredictor, apply_pruning
from agedataset import AgeDataset, get_transforms, load_dataset

import copy

In [2]:


def calculate_metrics(outputs, ages, scale_params):
    """Calculate MAE and accuracy metrics"""
    with torch.no_grad():
        pred_ages = outputs.cpu().numpy().squeeze() * (scale_params['max_age'] - scale_params['min_age']) + scale_params['min_age']
        true_ages = ages.cpu().numpy() * (scale_params['max_age'] - scale_params['min_age']) + scale_params['min_age']
        
        mae = float(np.mean(np.abs(pred_ages - true_ages)))
        accuracy_10 = float(np.mean(np.abs(pred_ages - true_ages) <= 10))
        accuracy_15 = float(np.mean(np.abs(pred_ages - true_ages) <= 15))
        
        return mae, accuracy_10, accuracy_15


def plot_training_history(history, save_path):
    """학습 히스토리 시각화"""
    plt.figure(figsize=(15, 5))
    
    # Loss plot
    plt.subplot(1, 3, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Val Loss')
    plt.title('Loss History')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    # MAE plot
    plt.subplot(1, 3, 2)
    plt.plot(history['train_mae'], label='Train MAE')
    plt.plot(history['val_mae'], label='Val MAE')
    plt.title('MAE History')
    plt.xlabel('Epoch')
    plt.ylabel('MAE (years)')
    plt.legend()
    
    # Accuracy plot
    plt.subplot(1, 3, 3)
    plt.plot(history['train_acc10'], label='Train Acc@10')
    plt.plot(history['val_acc10'], label='Val Acc@10')
    plt.plot(history['train_acc15'], label='Train Acc@15')
    plt.plot(history['val_acc15'], label='Val Acc@15')
    plt.title('Accuracy History')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.tight_layout()
    plt.savefig(os.path.join(save_path, 'training_history.png'))
    plt.close()

def train_teacher_model(model, train_loader, val_loader, device, scale_params, save_path, num_epochs=20):
    """Teacher 모델 학습"""
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)
    
    best_val_loss = float('inf')
    history = {
        'train_loss': [], 'val_loss': [], 
        'train_mae': [], 'val_mae': [],
        'train_acc10': [], 'val_acc10': [], 
        'train_acc15': [], 'val_acc15': []
    }
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0
        train_mae = 0
        train_acc10 = 0
        train_acc15 = 0
        
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} Training')
        for images, ages in train_bar:
            images, ages = images.to(device), ages.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.squeeze(), ages)
            loss.backward()
            optimizer.step()
            
            mae, acc10, acc15 = calculate_metrics(outputs, ages, scale_params)
            
            train_loss += float(loss.item())
            train_mae += mae
            train_acc10 += acc10
            train_acc15 += acc15
            
            train_bar.set_postfix({
                'loss': loss.item(),
                'MAE': mae,
                'Acc@10': f'{acc10*100:.1f}%',
                'Acc@15': f'{acc15*100:.1f}%'
            })
        
        # 검증
        model.eval()
        val_loss = 0
        val_mae = 0
        val_acc10 = 0
        val_acc15 = 0
        
        with torch.no_grad():
            for images, ages in val_loader:
                images, ages = images.to(device), ages.to(device)
                outputs = model(images)
                loss = criterion(outputs.squeeze(), ages)
                
                mae, acc10, acc15 = calculate_metrics(outputs, ages, scale_params)
                
                val_loss += float(loss.item())
                val_mae += mae
                val_acc10 += acc10
                val_acc15 += acc15
        
        # 평균 계산
        metrics = {
            'train_loss': train_loss / len(train_loader),
            'train_mae': train_mae / len(train_loader),
            'train_acc10': train_acc10 / len(train_loader),
            'train_acc15': train_acc15 / len(train_loader),
            'val_loss': val_loss / len(val_loader),
            'val_mae': val_mae / len(val_loader),
            'val_acc10': val_acc10 / len(val_loader),
            'val_acc15': val_acc15 / len(val_loader)
        }
        
        # 히스토리 업데이트
        for k, v in metrics.items():
            history[k].append(v)
        
        print(f'\nEpoch {epoch+1}:')
        print(f'Train - Loss: {metrics["train_loss"]:.4f}, MAE: {metrics["train_mae"]:.2f} years')
        print(f'Val   - Loss: {metrics["val_loss"]:.4f}, MAE: {metrics["val_mae"]:.2f} years')
        
        scheduler.step(metrics['val_loss'])
        
        if metrics['val_loss'] < best_val_loss:
            best_val_loss = metrics['val_loss']
            
            # 모델 저장
            save_dict = {
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch,
                'metrics': {k: float(v) for k, v in metrics.items()}
            }
            torch.save(save_dict, os.path.join(save_path, 'teacher_model_best.pth'))
            
            # 메트릭 저장
            with open(os.path.join(save_path, 'teacher_metrics.json'), 'w') as f:
                json.dump({
                    'best_epoch': epoch,
                    'metrics': {k: float(v) for k, v in metrics.items()}
                }, f, indent=4)
    
    # 학습 히스토리 시각화
    plot_training_history(history, save_path)
    return model, history

def train_student_model(teacher_model, student_model, train_loader, val_loader, device, 
                       scale_params, save_path, temperature=3.0, alpha=0.5, num_epochs=20):
    """Knowledge Distillation을 사용한 Student 모델 학습"""
    criterion_kd = nn.KLDivLoss(reduction='batchmean')
    criterion_task = nn.L1Loss()
    optimizer = optim.Adam(student_model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)
    
    teacher_model.eval()
    best_val_loss = float('inf')
    history = {
        'train_loss': [], 'val_loss': [], 
        'train_mae': [], 'val_mae': [],
        'train_acc10': [], 'val_acc10': [], 
        'train_acc15': [], 'val_acc15': []
    }
    
    for epoch in range(num_epochs):
        student_model.train()
        train_loss = 0
        train_mae = 0
        train_acc10 = 0
        train_acc15 = 0
        
        train_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} Training')
        for images, ages in train_bar:
            images, ages = images.to(device), ages.to(device)
            optimizer.zero_grad()
            
            # Teacher의 출력
            with torch.no_grad():
                teacher_outputs = teacher_model(images)
            
            # Student의 출력
            student_outputs = student_model(images)
            
            # Knowledge Distillation Loss
            loss_kd = criterion_kd(
                torch.log_softmax(student_outputs/temperature, dim=1),
                torch.softmax(teacher_outputs/temperature, dim=1)
            ) * (temperature * temperature)
            
            # Task-specific Loss
            loss_task = criterion_task(student_outputs.squeeze(), ages)
            
            # Total Loss
            loss = alpha * loss_kd + (1 - alpha) * loss_task
            loss.backward()
            optimizer.step()
            
            mae, acc10, acc15 = calculate_metrics(student_outputs, ages, scale_params)
            
            train_loss += float(loss.item())
            train_mae += mae
            train_acc10 += acc10
            train_acc15 += acc15
            
            train_bar.set_postfix({
                'loss': loss.item(),
                'MAE': mae,
                'Acc@10': f'{acc10*100:.1f}%',
                'Acc@15': f'{acc15*100:.1f}%'
            })
        
        # 검증
        student_model.eval()
        val_loss = 0
        val_mae = 0
        val_acc10 = 0
        val_acc15 = 0
        
        with torch.no_grad():
            for images, ages in val_loader:
                images, ages = images.to(device), ages.to(device)
                outputs = student_model(images)
                loss = criterion_task(outputs.squeeze(), ages)
                
                mae, acc10, acc15 = calculate_metrics(outputs, ages, scale_params)
                
                val_loss += float(loss.item())
                val_mae += mae
                val_acc10 += acc10
                val_acc15 += acc15
        
        # 평균 계산
        metrics = {
            'train_loss': train_loss / len(train_loader),
            'train_mae': train_mae / len(train_loader),
            'train_acc10': train_acc10 / len(train_loader),
            'train_acc15': train_acc15 / len(train_loader),
            'val_loss': val_loss / len(val_loader),
            'val_mae': val_mae / len(val_loader),
            'val_acc10': val_acc10 / len(val_loader),
            'val_acc15': val_acc15 / len(val_loader)
        }
        
        # 히스토리 업데이트
        for k, v in metrics.items():
            history[k].append(v)
        
        print(f'\nEpoch {epoch+1}:')
        print(f'Train - Loss: {metrics["train_loss"]:.4f}, MAE: {metrics["train_mae"]:.2f} years')
        print(f'Val   - Loss: {metrics["val_loss"]:.4f}, MAE: {metrics["val_mae"]:.2f} years')
        
        scheduler.step(metrics['val_loss'])
        
        if metrics['val_loss'] < best_val_loss:
            best_val_loss = metrics['val_loss']
            
            # 모델 저장
            save_dict = {
                'model_state_dict': student_model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'epoch': epoch,
                'metrics': {k: float(v) for k, v in metrics.items()}
            }
            torch.save(save_dict, os.path.join(save_path, 'student_model_best.pth'))
            
            # 메트릭 저장
            with open(os.path.join(save_path, 'student_metrics.json'), 'w') as f:
                json.dump({
                    'best_epoch': epoch,
                    'metrics': {k: float(v) for k, v in metrics.items()}
                }, f, indent=4)
    
    # 학습 히스토리 시각화
    plot_training_history(history, save_path)
    return student_model, history

def evaluate_model_performance(model, data_loader, device, scale_params):
    """모델의 정확도 및 MAE 평가"""
    model.eval()
    predictions = []
    true_ages = []
    
    with torch.no_grad():
        for images, ages in tqdm(data_loader, desc="Evaluating"):
            images = images.to(device)
            outputs = model(images)
            
            # 스케일 복원
            pred_ages = outputs.cpu().numpy().squeeze() * (scale_params['max_age'] - scale_params['min_age']) + scale_params['min_age']
            true_ages_batch = ages.numpy() * (scale_params['max_age'] - scale_params['min_age']) + scale_params['min_age']
            
            predictions.extend(pred_ages)
            true_ages.extend(true_ages_batch)
    
    predictions = np.array(predictions)
    true_ages = np.array(true_ages)
    
    mae = mean_absolute_error(true_ages, predictions)
    accuracy_10 = np.mean(np.abs(predictions - true_ages) <= 10)
    accuracy_15 = np.mean(np.abs(predictions - true_ages) <= 15)
    
    return {
        'mae': float(mae),
        'accuracy_10': float(accuracy_10),
        'accuracy_15': float(accuracy_15),
        'predictions': predictions,
        'true_ages': true_ages
    }

def compare_models(models, val_loader, device, scale_params, save_path='results'):
    """여러 모델의 성능 비교"""
    os.makedirs(save_path, exist_ok=True)
    results = {}
    
    for name, model in models.items():
        print(f"\nEvaluating {name} model...")
        model = model.to(device)
        
        # 추론 시간 측정
        start_time = time.time()
        performance = evaluate_model_performance(model, val_loader, device, scale_params)
        inference_time = (time.time() - start_time) / len(val_loader.dataset)
        
        # 모델 크기 계산 (MB)
        model_size = sum(p.nelement() * p.element_size() for p in model.parameters()) / (1024 * 1024)
        
        results[name] = {
            'mae': float(performance['mae']),
            'accuracy_10': float(performance['accuracy_10']),
            'accuracy_15': float(performance['accuracy_15']),
            'inference_time': float(inference_time),
            'model_size': float(model_size),
            'predictions': performance['predictions'],
            'true_ages': performance['true_ages']
        }
        
        print(f"\n{name} Model Results:")
        print(f"Mean Absolute Error: {performance['mae']:.2f} years")
        print(f"Accuracy within 10 years: {performance['accuracy_10']*100:.2f}%")
        print(f"Accuracy within 15 years: {performance['accuracy_15']*100:.2f}%")
        print(f"Average Inference Time: {inference_time*1000:.2f} ms per image")
        print(f"Model Size: {model_size:.2f} MB")
        
        # 산점도 그리기
        plt.figure(figsize=(10, 6))
        plt.scatter(performance['true_ages'], performance['predictions'], alpha=0.5)
        plt.plot([0, 100], [0, 100], 'r--')
        plt.xlabel('True Age')
        plt.ylabel('Predicted Age')
        plt.title(f'{name} Model: Predicted vs True Age')
        plt.savefig(os.path.join(save_path, f'{name.lower()}_scatter.png'))
        plt.close()
    
    # 모델 간 비교 그래프
    metrics = ['mae', 'accuracy_10', 'accuracy_15', 'inference_time', 'model_size']
    for metric in metrics:
        plt.figure(figsize=(8, 5))
        values = [results[model][metric] for model in results]
        plt.bar(results.keys(), values)
        plt.title(f'Model Comparison: {metric}')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(os.path.join(save_path, f'comparison_{metric}.png'))
        plt.close()
    
    # 결과 JSON 저장
    
    with open(os.path.join(save_path, 'comparison_results.json'), 'w') as f:
        json_results = {name: {k: float(v) if isinstance(v, (np.float32, np.float64)) else v 
                              for k, v in metrics.items() if not isinstance(v, np.ndarray)}
                       for name, metrics in results.items()}
        json.dump(json_results, f, indent=4)
    
    return results



In [ ]:
if __name__ == '__main__':
    # 설정
    DATA_PATH = '/Users/koallako/cau/4-2/Multimodal/embeddedAI/agedata/2400_dataset'
    SAVE_PATH = 'models'
    BATCH_SIZE = 32
    NUM_EPOCHS = 20
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 데이터 로드
    image_paths, ages, scale_params = load_dataset(DATA_PATH)
    train_paths, val_paths, train_ages, val_ages = train_test_split(
        image_paths, ages, test_size=0.2, random_state=42
    )
    
    # 데이터셋 및 데이터로더 생성
    train_transform, val_transform = get_transforms()
    train_dataset = AgeDataset(train_paths, train_ages, train_transform)
    val_dataset = AgeDataset(val_paths, val_ages, val_transform)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
    
    # Teacher 모델 학습
    print("\nTraining Teacher Model...")
    teacher_model = AgePredictor(is_student=False).to(DEVICE)
    teacher_model, teacher_history = train_teacher_model(
        teacher_model, train_loader, val_loader, DEVICE, scale_params, SAVE_PATH, NUM_EPOCHS
    )
    
    # Student 모델 학습 (Knowledge Distillation)
    print("\nTraining Student Model...")
    student_model = AgePredictor(is_student=True).to(DEVICE)
    student_model, student_history = train_student_model(
        teacher_model, student_model, train_loader, val_loader, 
        DEVICE, scale_params, SAVE_PATH, NUM_EPOCHS
    )
    
    # 다양한 프루닝 비율로 프루닝 및 저장
    pruning_ratios = [0.3, 0.5, 0.7]
    for ratio in pruning_ratios:
        print(f"\nCreating Pruned Model with {ratio*100}% pruning...")
        pruned_model = AgePredictor(is_student=False).to(DEVICE)
        pruned_model.load_state_dict(teacher_model.state_dict())
        pruned_model = apply_pruning(pruned_model, ratio)
        
        # 프루닝된 모델 저장
        pruned_save_path = os.path.join(SAVE_PATH, f'age_model_pruned_{int(ratio*100)}.pth')
        pruned_save_dict = {
            'model_state_dict': pruned_model.state_dict(),
            'scale_params': scale_params,
            'pruning_ratio': ratio
        }
        torch.save(pruned_save_dict, pruned_save_path)
        print(f"Pruned model saved successfully to {pruned_save_path}")
    
    # 모델 비교
    print("\nComparing Models...")
    models = {
        'Teacher': teacher_model,
        'Student': student_model,
    }
    for ratio in pruning_ratios:
        pruned_model_path = os.path.join(SAVE_PATH, f'age_model_pruned_{int(ratio*100)}.pth')
        pruned_model_loaded = AgePredictor(is_student=False)  # 동일한 모델 구조 생성
        pruned_checkpoint = torch.load(pruned_model_path, map_location=DEVICE)
        pruned_model_loaded.load_state_dict(pruned_checkpoint['model_state_dict'])
        pruned_model_loaded = pruned_model_loaded.to(DEVICE)
        models[f'Pruned ({int(ratio*100)}%)'] = pruned_model_loaded

    comparison_results = compare_models(models, val_loader, DEVICE, scale_params)


Loading dataset...

Training Teacher Model...


Epoch 1/20 Training: 100%|██████████| 75/75 [00:06<00:00, 11.31it/s, loss=0.174, MAE=17, Acc@10=43.8%, Acc@15=59.4%]  



Epoch 1:
Train - Loss: 0.2215, MAE: 21.71 years
Val   - Loss: 0.1773, MAE: 17.38 years


Epoch 2/20 Training: 100%|██████████| 75/75 [00:06<00:00, 11.47it/s, loss=0.2, MAE=19.6, Acc@10=28.1%, Acc@15=46.9%]  



Epoch 2:
Train - Loss: 0.1723, MAE: 16.89 years
Val   - Loss: 0.1632, MAE: 15.99 years


...


Pruned (30%) Model Results:
Mean Absolute Error: 8.95 years
Accuracy within 10 years: 67.00%
Accuracy within 15 years: 80.50%
Average Inference Time: 1.25 ms per image
Model Size: 8.36 MB

Evaluating Pruned (50%) model...


Evaluating: 100%|██████████| 19/19 [00:00<00:00, 25.51it/s]



Pruned (50%) Model Results:
Mean Absolute Error: 9.24 years
Accuracy within 10 years: 64.83%
Accuracy within 15 years: 79.00%
Average Inference Time: 1.25 ms per image
Model Size: 8.36 MB

Evaluating Pruned (70%) model...


Evaluating: 100%|██████████| 19/19 [00:00<00:00, 25.36it/s]



Pruned (70%) Model Results:
Mean Absolute Error: 9.64 years
Accuracy within 10 years: 61.67%
Accuracy within 15 years: 77.33%
Average Inference Time: 1.25 ms per image
Model Size: 8.36 MB
